In [1]:
# fix python path if working locally
# from utils import fix_pythonpath_if_working_locally

# fix_pythonpath_if_working_locally()

import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt

from darts import TimeSeries
from darts.utils.timeseries_generation import (
    gaussian_timeseries,
    linear_timeseries,
    sine_timeseries,
)
from darts.models import (
    RNNModel,
    TCNModel,
    TransformerModel,
    NBEATSModel,
    BlockRNNModel,
    VARIMA,
)
from darts.metrics import mape, smape, mae, rmse
from darts.dataprocessing.transformers import Scaler
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from darts.datasets import AirPassengersDataset, MonthlyMilkDataset, ElectricityDataset

import logging

logging.disable(logging.CRITICAL)

import warnings

warnings.filterwarnings("ignore")

%matplotlib inline

# for reproducibility
torch.manual_seed(1)
np.random.seed(1)

In [2]:
df1 = pd.read_csv('archive/TripA01.csv', sep=';', encoding="unicode_escape")
df2 = pd.read_csv('archive/TripA02.csv', sep=';', encoding="unicode_escape")
# df3 = pd.read_csv('archive/TripA03.csv', sep=';', encoding="unicode_escape")
# df4 = pd.read_csv('archive/TripA04.csv', sep=';', encoding="unicode_escape")

In [ ]:
df1.set_index("Time [s]", inplace=False)
df2.set_index("Time [s]", inplace=False)

start_time = '2020-01-01'  # Example start date
df1.index = pd.date_range(start=start_time, periods=len(df1), freq='S')
df2.index = pd.date_range(start=start_time, periods=len(df2), freq='S')

df1.index.freq = 'S'
df2.index.freq = 'S'

velo_ts1 = TimeSeries.from_dataframe(df1, value_cols='Velocity [km/h]')
accel_ts1 = TimeSeries.from_dataframe(df1, value_cols='Longitudinal Acceleration [m/s^2]')
soc_ts1 = TimeSeries.from_dataframe(df1, value_cols='SoC [%]')


velo_ts2 = TimeSeries.from_dataframe(df2, value_cols='Velocity [km/h]')
accel_ts2 = TimeSeries.from_dataframe(df2, value_cols='Longitudinal Acceleration [m/s^2]')
soc_ts2 = TimeSeries.from_dataframe(df2, value_cols='SoC [%]')

batt_covar1 = velo_ts1.stack(accel_ts1)
batt_covar2 = velo_ts2.stack(accel_ts2)

scaler = Scaler()
batt_covar1 = scaler.fit_transform(batt_covar1)
batt_covar2 = scaler.fit_transform(batt_covar2)
batt_covar1.plot()

soc_ts1 = scaler.fit_transform(soc_ts1)
soc_ts2 = scaler.fit_transform(soc_ts2)

In [ ]:
soc_train1, soc_valid1 = soc_ts1[:-4000], soc_ts1[-4000:]
batt_covar_train1, batt_covar_valid1 = batt_covar1[:-4000], batt_covar1[-4000:]

soc_train2, soc_valid2 = soc_ts2[:-6000], soc_ts2[-6000:]
batt_covar_train2, batt_covar_valid2 = batt_covar2[:-6000], batt_covar2[-6000:]

In [ ]:
model_pastcov = BlockRNNModel(
    model="LSTM",
    input_chunk_length=600,
    output_chunk_length=60,
    n_epochs=10,
    random_state=0,
)

In [ ]:
model_pastcov.fit(
    series=[soc_train1, soc_train2],
    past_covariates=[batt_covar_train1, batt_covar_train2],
    verbose=True,
);

In [ ]:
pred_cov = model_pastcov.predict(n=4000, series=soc_train1, past_covariates=batt_covar1)

soc_ts1.plot(label="actual")
pred_cov.plot(label="forecast")
plt.legend()
print("RMSE = {:.2f}%".format(rmse(soc_ts1, pred_cov)))